## Make the figure #2
- Show our dataset 
- Conj / G.T / patch / patch G.T
- HRF / G.T / patch / patch G.T



In [66]:
import numpy as np
import os
import configparser
import sys
import imgaug as ia
import imgaug.augmenters as iaa
sys.path.insert(0, './lib/')
from help_functions import *
from lib.model_lib import *
from pre_processing import my_preprocessing
from pre_processing import augmentations
import random
import cv2

def data_consistency_check(imgs,masks):
    assert(len(imgs.shape)==len(masks.shape))
    assert(imgs.shape[0]==masks.shape[0])
    assert(imgs.shape[2]==masks.shape[2])
    assert(imgs.shape[3]==masks.shape[3])
    assert(masks.shape[1]==1)
    assert(imgs.shape[1]==1 or imgs.shape[1]==3)

In [67]:
config = configparser.RawConfigParser()
config.read('configuration.txt')
#patch to the datasets
path_data = config.get('data paths', 'path_local')
#Experiment name
save_folder = config.get('experiment name','result_save_path')
save_folder = 'result/'+save_folder
name_experiment = config.get('experiment name', 'name')

#training settings
num_epochs = int(config.get('training settings', 'num_epochs'))
batch_size = int(config.get('training settings', 'batch_size'))

In [112]:

def draw_figure2(img_path,groudTruth_path,
                      patch_height,patch_width,
                      num_subimgs,inside_FOV, save_path, train_masking_data = False):
    
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
    else:
        print('already exist the folder in this path : {}'.format(save_path))
    border_masking_flag = False
    if train_masking_data != False:
        train_border_masking = load_hdf5(train_masking_data)
        border_masking_flag = True
        print('[DEBUG] shape of border masking : ',np.shape(train_border_masking))
        


    
    train_imgs_original = load_hdf5(img_path)
    train_masks = load_hdf5(groudTruth_path)
    train_imgs_original = train_imgs_original[:,:,:,:]
    train_masks = train_masks[:,:,:,:]
    
    temp_original = np.transpose(train_imgs_original,(0,2,3,1))
    temp_gt = np.transpose(train_masks,(0,2,3,1))

    for i in range(np.shape(train_imgs_original)[0]):
        cv2.imwrite('./'+save_path+'/raw_data/conj/'+str(i)+'.png', temp_original[i])
        cv2.imwrite('./'+save_path+'/raw_data/groundTruth/'+str(i)+'.png', temp_gt[i])


    
    visualize(group_images(train_imgs_original[0:20,:,:,:],5),'./'+save_path+'/imgs_train')  #check original imgs train
    
    train_imgs = my_preprocessing(train_imgs_original)
    #temp2_totimg = visualize(group_images(train_imgs[:,:,:,:],5),'./'+ save_path+'/preprocessed')
    temp_pre = np.transpose(train_imgs,(0,2,3,1))
    for i in range(np.shape(train_imgs_original)[0]):
        cv2.imwrite('./'+save_path+'/raw_data/preprocessing/'+str(i)+'.png', temp_pre[i]*255)

    train_masks = train_masks/255.
    
    #check masks are within 0-1
    print(np.max(train_masks))
    assert(np.min(train_masks)==0 and np.max(train_masks)==1)

    print ('\n[get_data_training] train images/masks shape : {}'.format(train_imgs.shape))
    print ('[get_data_training] train images range (min-max) [{} , {}] '.format(str(np.min(train_imgs)),str(np.max(train_imgs))))
    print ('[get_data_training] train masks are within 0-1\n')
    
    for i in range(np.shape(train_imgs_original)[0]):
        temp_imgs = np.expand_dims(train_imgs[i,:,:,:],0)
        temp_gt = np.expand_dims(train_masks[i,:,:,:],0)
        
        patches_imgs_train, patches_masks_train = extract_random(temp_imgs,temp_gt,patch_height,patch_width,num_subimgs, inside = inside_FOV,masking_flag = border_masking_flag)
        
        print('patch shape : ',np.shape(patches_imgs_train))
        print('patch gt shape : ', np.shape(patches_masks_train))
        visualize(group_images(patches_imgs_train[200:300,:,:,:]* 255,5),'./'+save_path+'/raw_data/patch/'+str(i)+'_train_patch_img')
        visualize(group_images(patches_masks_train[200:300,:,:,:]* 255,5),'./'+save_path+'/raw_data/patch_gt/'+str(i)+'_train_patch_img')

    #patches_imgs_train, patches_masks_train = extract_dynamic_random(train_imgs,train_masks,patch_height,patch_width,num_subimgs,inside_FOV)
    #data_consistency_check(patches_imgs_train, patches_masks_train)
    #extract the TRAINING patches from the full images
    #extraact the random patches for data augmentation
    
    print ('\n[get_data_training] train PATCHES images/masks shape : {}'.format(patches_imgs_train.shape))
    print ('[get_data_training] train PATCHES images range (min-max): ' +str(np.min(patches_imgs_train)) +' - '+str(np.max(patches_imgs_train)))
    
    print('[get_data_training] patches_imgs_train : {}'.format(patches_imgs_train.shape))
    #visualize(group_images(patches_imgs_train[0:50,:,:,:]* 255,5),'./'+save_path+'/train_patch_img')
    
    #patches_imgs_train = augmentations(patches_imgs_train,0.4)
    print('[augmentation] patches_imgs_train : {}'.format(patches_imgs_train.shape))
    #temp3_totimg = visualize(group_images(patches_imgs_train[0:50,:,:,:],5),'./'+save_path+'/augmented_patch_img')
    data_consistency_check(patches_imgs_train, patches_masks_train)
    
    return patches_imgs_train, patches_masks_train


def extract_random(full_imgs,full_masks, patch_h,patch_w, num_patches, inside=True, ratio =1,masking_flag = False, border_masking = None):
    num_patches = int(num_patches * ratio)
    '''
    if (num_patches % full_imgs.shape[0] != 0): #나머지가 0
        print ("[extract random] num_patches: please enter a multiple of 20")
        print('[extract randon] num_patches : {} , full imgs shape {}'.format(num_patches, full_imgs.shape))
        exit()'''
    #why..?
    assert (len(full_imgs.shape)==4 and len(full_masks.shape)==4)  #4D arrays (Tensor)
    assert (full_imgs.shape[1]==1 or full_imgs.shape[1]==3)  #check the channel is 1 or 3
    assert (full_masks.shape[1]==1)   #masks only black and white
    assert (full_imgs.shape[2] == full_masks.shape[2] and full_imgs.shape[3] == full_masks.shape[3]) #두 data의 사이즈는 같아야 한다.
    
    
    # each patches
    
    patches = np.empty((num_patches,full_imgs.shape[1], patch_h, patch_w),dtype = np.float32)
    patches_masks = np.empty((num_patches, full_masks.shape[1], patch_h, patch_w))
    
    print('[extract random] patches shape : {}'.format(patches.shape))
    
    img_h = full_imgs.shape[2]
    img_w = full_imgs.shape[3]
    
    patch_per_img =int(num_patches / full_imgs.shape[0]) #ex, 48 / 20
    print('[extract random] patches per full image : {}'.format(patch_per_img))
    iter_total =0 #iter over the total number of patches (num_patches)
    
    for i in range(full_imgs.shape[0]):  # loop, number of image samples
        k =0
        temppp_cnt = 0
        
        
        while k<patch_per_img: #iter, number of patch per img
            x_center = random.randint(0+int(patch_w/2) , img_w-int(patch_w/2))
            #print('[extract random] x_center : {}'.format(x_center))
            y_center = random.randint(0+int(patch_w/2) , img_h-int(patch_w/2))
            
            if border_masking == True:
                border_patch = border_masking[i,0, y_center-int(patch_h/2):y_center+int(patch_h/2),x_center-int(patch_w/2):x_center+int(patch_w/2)]
                border_check = np.sum(border_patch)
            
            if  border_masking == 0.0 or  border_masking ==0:
                pass
                    
            patch = full_imgs[i,:,y_center-int(patch_h/2):y_center+int(patch_h/2),x_center-int(patch_w/2):x_center+int(patch_w/2)] #cropping
            patch_mask = full_masks[i,:,y_center-int(patch_h/2):y_center+int(patch_h/2),x_center-int(patch_w/2):x_center+int(patch_w/2)]
            
            
            patches[iter_total]=patch
            patches_masks[iter_total]=patch_mask
            iter_total +=1   #total
            k+=1  #per full_img
            
    return patches, patches_masks

In [113]:
def visualize(data,filename): # plot image
    assert (len(data.shape)==3) #height*width*channels
    img = None
    print('data shape : ',data.shape)
    if data.shape[2]==1:  #in case it is black and white
        data = np.reshape(data,(data.shape[0],data.shape[1]))
    if np.max(data)>1:
        img = Image.fromarray(data.astype(np.uint8))   #the image is already 0-255
    else:
        img = Image.fromarray((data*255).astype(np.uint8))  #the image is between 0-1
    print(img)
    img.save(filename + '.png')
    print('file name : ',filename)
    return img


#group a set of images row per columns
def group_images(data,per_row):
    print('[group images func] prev data shape  :', data.shape)
    assert (data.shape[0]%per_row==0)
    assert (data.shape[1]==1 or data.shape[1]==3)
    data = np.transpose(data,(0,2,3,1))  #corect format for imshow, make tensor, tensorflow backend
    print('[group images func] after data shape : ', data.shape)

    all_stripe = []
    for i in range(int(data.shape[0]/per_row)):
        stripe = data[i*per_row]
        for k in range(i*per_row+1, i*per_row+per_row):
            stripe = np.concatenate((stripe,data[k]),axis=1)
        all_stripe.append(stripe)
    totimg = all_stripe[0]
    print('[group images func] first total image : ', totimg.shape)

    for i in range(0,len(all_stripe)):
        totimg = np.concatenate((totimg,all_stripe[i]),axis=0)        
    print('[group images func] final total image : ', totimg.shape)

    return totimg

In [114]:
save_figure = './figure/figure2/'

patches_imgs_train, patches_masks_train = draw_figure2(
    img_path = path_data + config.get('data paths', 'CONJ_train_imgs_original'),
    groudTruth_path = path_data + config.get('data paths', 'CONJ_train_groundTruth'),  #masks
    patch_height = int(config.get('data attributes', 'patch_height')),
    patch_width = int(config.get('data attributes', 'patch_width')),
    num_subimgs = 3000,
    inside_FOV = config.getboolean('training settings', 'inside_FOV'), #select the patches only inside the FOV  (default == True)
    save_path = save_figure)

already exist the folder in this path : ./figure/figure2/
[group images func] prev data shape  : (20, 3, 900, 1100)
[group images func] after data shape :  (20, 900, 1100, 3)
[group images func] first total image :  (900, 5500, 3)
[group images func] final total image :  (4500, 5500, 3)
data shape :  (4500, 5500, 3)
<PIL.Image.Image image mode=RGB size=5500x4500 at 0x7F2F29E1C908>
file name :  ././figure/figure2//imgs_train
[DEBUG] normalize shape :  (60, 1, 900, 1100)
[DEBUG] i normalize shape :  (1, 900, 1100)
1.0

[get_data_training] train images/masks shape : (60, 1, 900, 1100)
[get_data_training] train images range (min-max) [0.0 , 1.0] 
[get_data_training] train masks are within 0-1

[extract random] patches shape : (3000, 1, 64, 64)
[extract random] patches per full image : 3000
patch shape :  (3000, 1, 64, 64)
patch gt shape :  (3000, 1, 64, 64)
[group images func] prev data shape  : (100, 1, 64, 64)
[group images func] after data shape :  (100, 64, 64, 1)
[group images func] f